In [ ]:
from transformers import AutoModel,AutoTokenizer , Trainer,TrainingArguments,BitsAndBytesConfig,pipeline
from peft import get_peft_model,LoraConfig
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd

cache_dir='/proj/ciptmp/ix05ogym/.cache/'


In [ ]:
mydata = pd.read_pickle('input_text.pkl')
mydata.pop('label')
mydata['labels'].unique()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli" , cache_dir=cache_dir)
#x = tokenizer("[[xpath]]")
#tokenizer.convert_ids_to_tokens(x['input_ids'])
#tokenizer.additional_special_tokens

In [ ]:
#weblinx_dataset = datasets.load_dataset('',cache_dir=cache_dir)
#weblinx_dataset.keys()


In [ ]:
model_name = "FacebookAI/roberta-large-mnli"
#config = BitsAndBytesConfig(
    #load_in_4bit=True,

    #load_in_8bit=True,
    
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_compute_dtype=torch.bfloat16,
    #)


model = AutoModel.from_pretrained(model_name,
                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),
                          #load_in_8bit=True,
                          cache_dir=cache_dir).cuda()


print(model)


"""
#https://huggingface.co/docs/transformers/main/en/quantization/hqq
try:
 model_hqq = HQQModelForCausalLM.from_quantized(model_name, device_map='auto',cache_dir=cache_dir).half()#.to(torch.bfloat16) #.half()
except: 
 model_hqq = AutoHQQHFModel.from_quantized(model_name,cache_dir=cache_dir)
 
"""
 



In [ ]:
pip = pipeline('zero-shot-classification',model = model)#pipeline_class=['email','firstname','lastname','cv'])
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
o = pip(sequence_to_classify, candidate_labels)

print(o)

In [ ]:
#model = model.eval()
#input_ids = tokenizer("create an html apply button?", return_tensors='pt').to('cuda')["input_ids"]
#outputs = model.generate(input_ids, max_new_tokens=216)
#tokenizer.decode(outputs[0])